## Please provide your inputs as followed:

- **input_sdate**: The start date of your period of interest in the format **dd-mm-YYYY**
- **input_edate**: The end date of your period of interestin the format **dd-mm-YYYY**
- **geometry**: The region of interest. Please provide this in a **bounding box format** (e.g. [10, -5, 25, 20])

In [ ]:
start_date = "2023 08 20"
end_date = "2023 08 25"
bbox = [112.70505, -44.52755, 154.38241, -11.29524]
country = "Australia"

## Library imports 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import netCDF4 as nc4
import xarray as xr
import fsspec
import numpy as np
import xarray as xr
import planetary_computer
import pystac_client
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, time
import json

## Starting up PySTAC client

In [ ]:
# Initialize PySTAC client for data query
planetary_computer.set_subscription_key("c27669c4bdec434d804e2bd738cb16fc")
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

## User Input

### Processing of user input

In [ ]:
# Function to convert date format 
def convert_format_date(input_date):
    correct_formats = ["%d %m %Y", "%Y %m %d", "%d/%m/%Y", "%Y/%m/%d", "%d-%m-%Y", "%Y-%m-%d"]
    
    for format_str in correct_formats:
        try:
            date_obj = datetime.strptime(input_date, format_str)
            formatted_date = date_obj.strftime("%Y-%m-%d")
            return formatted_date
        except ValueError: # Raised if input format is not compatible with set standard 
            pass
    
    raise ValueError("Invalid data format")
    
# Convert user start date format
try:
    start_date = convert_format_date(start_date)
except ValueError:
    print("Invalid start date format. Please check the acceptable formats")
            
# Convert user end date format
try:
    end_date = convert_format_date(end_date)
except ValueError:
    print("Invalid end date format. Please check the acceptable formats")

date_period = start_date + "/" + end_date 
print(date_period)

## Search for product

### Search based on country input

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) # Get geopandas in-built naturalearth_lowres dataset

# Extract coordinates of specified country and load into a JSON object
ROI = world[world["name"] == country]
gjson = json.loads(ROI.geometry.to_json())
coordinates = gjson["features"][0]["geometry"]

In [ ]:
search = catalog.search(
    collections="sentinel-5p-l2-netcdf",
    intersects=coordinates,
    datetime=date_period,
    query={"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "ch4"}},
)
items = search.item_collection()

print(len(items))

### Search based on bbox

In [ ]:
search = catalog.search(
    collections="sentinel-5p-l2-netcdf",
    bbox=bbox,
    datetime=date_period,
    query={"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "ch4"}},
)
items = search.item_collection()

print(len(items))

## Process queried data into a dataset

In [ ]:
item_links = [item.assets['ch4'].href for item in items]
item_links

f = fsspec.open_files(item_links)
f = [file.open() for file in f]


In [ ]:
datasets = [xr.open_dataset(nc_file, group="PRODUCT", engine="h5netcdf") for nc_file in f]

In [ ]:
# Initialize the map
fig, ax = plt.subplots(1, 1, figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})
continent_borders = world.dissolve(by='continent')
continent_borders.boundary.plot(ax=ax, linewidth=1, color='black')

for item in f: # For each opened item in query
    ds = xr.open_dataset(item, group="PRODUCT", engine="h5netcdf") # Create a dataset
    
    for time in range(ds.dims["time"]): # For each time within a dataset
        
        # Extract the relevant data (assuming the variable name is 'methane_mixing_ratio_bias_corrected')
        data = ds['methane_mixing_ratio_bias_corrected'][0, :, :] # 
        #print(data.values)
        lon = ds['longitude'].values.squeeze()
        lat = ds['latitude'].values.squeeze()

        # Calculate vmin and vmax for color normalization
        vmin, vmax = np.nanpercentile(data, [1, 99])

        # Plot the data
        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        colormesh = ax.pcolormesh(lon, lat, data.values, cmap="Spectral", norm=norm, transform=ccrs.PlateCarree(), alpha=0.9, rasterized=True)
        
fig.colorbar(colormesh, pad=0.05, shrink=0.35, label="methane (mol/m2)")

ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])

plt.show()

In [ ]:
datasets[0].time.values[0]

In [ ]:
ds = datasets[dataset.time == np.datetime64("2023-08-25") for dataset in datasets]
xr.merge

---

## END OF WORKING PIPELINE

## IGNORE BOTTOM SECTION FOR THE MOMENT

---

In [ ]:
ds = xr.open_mfdataset(f, group="PRODUCT", engine="h5netcdf", combine='by_coords') 
ds

In [ ]:
ds2 = xr.open_mfdataset(f, group="PRODUCT", engine="h5netcdf", concat_dim="t", combine='nested') 
ds2

In [ ]:
varname = "methane_mixing_ratio_bias_corrected"

for time in range(ds.dims["time"]):    
    data = ds[varname][time, :, :]
    vmin, vmax = np.nanpercentile(data, [1, 99])

    # methane product (NaN locations are transparent)
    lon = ds2["longitude"][time].values.squeeze()
    lat = ds2["latitude"][time].values.squeeze()
    methane = data.values
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    # colormesh = ax.pcolormesh(lon, lat, data.values, cmap="Spectral", norm=norm, transform=ccrs.PlateCarree())

    print(lon)

In [ ]:
varname = "methane_mixing_ratio_bias_corrected"
for darr_idx in range(ds2.dims["t"]):
    for time in range(ds2.dims["time"]):    
        data2 = ds2[varname][time, :, :]
        vmin2, vmax2 = np.nanpercentile(data2, [1, 99])

        # methane product (NaN locations are transparent)
        lon2 = ds2["longitude"][darr_idx][time].values.squeeze()
        lat2 = ds2["latitude"][darr_idx][time].values.squeeze()
        methane2 = data2.values
        norm2 = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        # colormesh2 = ax.pcolormesh(lon2, lat2, data2.values, cmap="Spectral", norm=norm2, transform=ccrs.PlateCarree())

    print(lon2)

In [ ]:
varname = "methane_mixing_ratio_bias_corrected"
data = ds[varname][0, :, :]
vmin, vmax = np.nanpercentile(data, [1, 99])

# methane product (NaN locations are transparent)
lon = ds["longitude"][0].values.squeeze()
lat = ds["latitude"][0].values.squeeze()
methane = data.values
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

In [ ]:
data2 = ds2[varname][0, :, :]
vmin2, vmax2 = np.nanpercentile(data2, [1, 99])

# methane product (NaN locations are transparent)
lon2 = ds2["longitude"][0].values.squeeze()
lat2 = ds2["latitude"][0].values.squeeze()
methane2 = data2.values
norm2 = matplotlib.colors.Normalize(vmin=vmin2, vmax=vmax2)

In [ ]:
data2

In [ ]:
ds2["longitude"][0][0].values

## Plot Base Map & Concentration

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) # Get geopandas in-built naturalearth_lowres dataset
continent_borders = world.dissolve(by='continent')

fig, ax = plt.subplots(1, 1, figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

if country != "":  
    specific_country = world[world['name'].str.strip() == country] 
    
    if not specific_country.empty:
        
        specific_country.boundary.plot(ax=ax, linewidth=2, color='black')
        
        plt.title(f"Map Highlighting {country}")
        
        ax.set_xlim(specific_country.total_bounds[0], specific_country.total_bounds[2])
        ax.set_ylim(specific_country.total_bounds[1], specific_country.total_bounds[3])
elif not not bbox:
    ax.set_xlim(bbox[0], bbox[2])
    ax.set_ylim(bbox[1], bbox[3])
else:
    print(f"Country '{country}' not found in the dataset.")
    world.boundary.plot(ax=ax, linewidth=1, color='black')
    plt.title("World Map")

ax.set_title("")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
        
for darray in range(ds2.dims["t"]):
    for time in range(ds2.dims["time"]):    
        data = ds[varname][time, :, :]
        vmin, vmax = np.nanpercentile(data, [1, 99])

        # methane product (NaN locations are transparent)
        lon = ds["longitude"][time].values.squeeze()
        lat = ds["latitude"][time].values.squeeze()
        methane = data.values
        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        
        colormesh = ax.pcolormesh(lon, lat, data.values, cmap="Spectral", norm=norm, transform=ccrs.PlateCarree())
fig.colorbar(colormesh2, pad=0.05, shrink=0.35, label="methane (mol/m2)")
plt.show()

In [ ]:
world[world["name"]=="Australia"].boundary

## Plot Time Series